<a href="https://colab.research.google.com/github/naotaqe/git_tutorial/blob/master/fall_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [228]:
!pip install -q pytorch-lightning
!pip install -q torchmetrics


import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torchvision import transforms, datasets
import torchmetrics
from torchmetrics.functional import accuracy

from torchvision.models import resnet18
from torchvision.models import vgg16_bn
from torchvision.models import mobilenet_v2

from torchmetrics.functional import accuracy
import torchsummary
from torchsummary import summary
from pytorch_lightning.loggers import CSVLogger

from PIL import Image

from google.colab import drive
drive.mount('/content/drive')

from torchvision import transforms, datasets

#transform を定義
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
#前処理とアノテーション（ラベル付け）
dataset = datasets.ImageFolder('/content/drive/MyDrive/abnormal_detect', transform)
print('下記がdatasetの中身です')
print(dataset)

# データセットの取得
train_val = dataset

# train と val に分割
pl.seed_everything(0)
n_train, n_val = round(len(dataset)*0.85), round(len(dataset)*0.15)
train, val = torch.utils.data.random_split(train_val, [n_train, n_val])

# バッチサイズの定義
batch_size = 5

# Data Loader を定義
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)

# ResNet を特徴抽出機として使用
feature = resnet18(pretrained=True)

#Netを定義する
class Net(pl.LightningModule):
    def __init__(self):
        super().__init__()

        self.feature = resnet18(pretrained=True)
        self.fc = nn.Linear(1000, 2)
    def forward(self, x):
        h = self.feature(x)
        h = self.fc(h)
        return h
    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('train_loss', loss, on_step=False, on_epoch=True)
        self.log('train_acc', accuracy(y.softmax(dim=-1), t), on_step=False, on_epoch=True)
        return loss
    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        self.log('val_acc', accuracy(y.softmax(dim=-1), t), on_step=False, on_epoch=True)
        return loss
    def test_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('test_loss', loss, on_step=False, on_epoch=True)
        self.log('test_acc', accuracy(y.softmax(dim=-1), t), on_step=False, on_epoch=True)
        return loss
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=0.01)
        return optimizer

# 学習の実行
pl.seed_everything(0)
net = Net()
logger = CSVLogger(save_dir='logs', name='my_exp')
trainer = pl.Trainer(max_epochs=3, gpus=1, deterministic=True, logger=logger)
trainer.fit(net, train_loader, val_loader)


# 学習済みモデルの保存
torch.save(net.state_dict(), 'fall.pt')

Global seed set to 0


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
下記がdatasetの中身です
Dataset ImageFolder
    Number of datapoints: 494
    Root location: /content/drive/MyDrive/abnormal_detect
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


Global seed set to 0
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type   | Params
-----------------------------------
0 | feature | ResNet | 11.7 M
1 | fc      | Linear | 2.0 K 
-----------------------------------
11.7 M    Trainable params
0         Non-trainable params
11.7 M    Total params
46.766    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 0


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [229]:
test = datasets.ImageFolder('/content/drive/MyDrive/abnormal_detect_test', transform)

test_loader = torch.utils.data.DataLoader(test)

results = trainer.test(test_dataloaders=test_loader)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:902: LightningDeprecationWarning: `trainer.test(test_dataloaders)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.test(dataloaders)` instead.
  "`trainer.test(test_dataloaders)` is deprecated in v1.4 and will be removed in v1.6."
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1394: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.model_checkpoint.last_model_path` to use the last model.
  f"`.{fn}(ckpt_path=None)` was called without a model."
Restoring states from the checkpoint path at logs/my_exp/version_1/checkpoints/epoch=2-step=251.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at logs/my_exp/version_1/checkpoints/epoch=2-step=251.ckpt


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 1.0, 'test_loss': 0.003831656649708748}
--------------------------------------------------------------------------------


In [239]:
# 写真の中に異常があるかどうかを判定する。
# ネットワークの準備
net = Net().cpu().eval()

net.load_state_dict(torch.load('fall.pt', map_location=torch.device('cpu')))

#imgを取得
img = Image.open('/content/IMG_3052.jpeg')
x =transform(img)
with torch.no_grad():
    y = net(x.unsqueeze(0))
print(y)

y = torch.argmax(net(x.unsqueeze(0)), dim=1).cpu().detach().numpy()

if y ==0:
  print('異常です！！')
else:
  print('正常です！！')

tensor([[ 4.1788, -4.2990]])
異常です！！
